# FSL-MRS fMRS fitting demo

## Setting up the problem

Data + basis set

## Setting up the fitting

Design matrix + model

## Performing the fit

Show code lines

See associated notebook (fast) or full fitting script

## Results at a single subject level

Load data from fit of windowed average

In [18]:
import pandas as pd

df_indep_stim = pd.read_csv('windowed_avg_results/sub0_stim.csv', index_col=0)
df_wavg_stim = pd.read_csv('windowed_avg_results/sub0_stim_wavg.csv', index_col=0)
df_dyn_sub0 = pd.read_csv('first_level_results/sub0/stim/free_parameters.csv', index_col=0)

Plot

In [39]:
design_matrix = pd.read_csv('designmat.csv',header=None).to_numpy()

def indiv_traces(metab, percent=True):
    p = df_dyn_sub0.filter(regex=metab, axis=0)['mean'].values
    val = np.dot(p, design_matrix.T)
    if percent:
        val = 100 * val/val.mean() - 100
    return val

In [49]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

met = 'Glu'

x_axis = np.arange(64)

colors = plt.get_cmap('tab10').colors
def color_t2s(tin, alpha=1):
    tscaled = [255*t for t in tin]
    return f'rgba({tscaled[0]}, {tscaled[1]}, {tscaled[2]}, {alpha})'
line_size = dict(idep=1, wavg=2, dyn=3)

mean_metab = df_indep_stim[met].mean()
def to_percent(x):
    return 100 * ((x/mean_metab) - 1)

indep_trace = go.Scatter(
    x=x_axis, y=to_percent(df_indep_stim[met]),
    mode='lines+markers',
    name=f'{met}: independent',
    legendgroup=met,
    line=dict(
        color=color_t2s(colors[0]),
        width=line_size['idep']),
                )

wav_trace = go.Scatter(
    x=x_axis, y=to_percent(df_wavg_stim[met]),
    mode='lines',
    name=f'{met}: window avg',
    legendgroup=met,
    line=dict(
        color=color_t2s(colors[1]),
        width=line_size['wavg']),
                )

regex = '_' + met.replace('+', '\+') + '_'
dyn_trace = go.Scatter(
    x=x_axis, y=indiv_traces(regex),
    mode='lines',
    name=f'{met}: dynamic',
    legendgroup=met,
    line=dict(
        color=color_t2s(colors[2]),
        width=line_size['dyn']),
                )

fig = go.Figure()
fig.add_trace(indep_trace)
fig.add_trace(wav_trace)
fig.add_trace(dyn_trace)
fig.update_layout(template='plotly_white')

## Second Level analysis